In [1]:
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

Retrieve Image Data

In [28]:
# TRAINING_DIR = os.path.join('lettuce', 'train')
# TESTING_DIR = os.path.join('lettuce', 'test')
# VALIDATION_DIR = os.path.join('lettuce', 'valid')
DIR = './FNNPK/'

Preprocessing Image

In [46]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)
# valid_datagen = ImageDataGenerator(rescale=1./255)
# 
# train_generator = train_datagen.flow_from_directory(
#     TRAINING_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )
# testing_generator = test_datagen.flow_from_directory(
#     TESTING_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )
# validation_generator = valid_datagen.flow_from_directory(
#     VALIDATION_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)
train_generator = train_datagen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = valid_datagen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 168 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


Build a custom callback

In [60]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor1, monitor2, target):
        super(CustomCallback, self).__init__()
        self.monitor1 = monitor1
        self.monitor2 = monitor2
        self.target = target
        
    def on_epoch_end(self, epoch, logs=None):
        current1 = logs[self.monitor1]
        current2 = logs[self.monitor2]
        if current1 is not None and current2 is not None:
            if current1 > self.target and current2 > self.target:
                print(f'{current1} and {current2} has reached {self.target}. Stopping training early.')
                self.model.stop_training = True
                
stop_when_80 = CustomCallback(monitor1='accuracy', monitor2='val_accuracy', target=0.8)

Build the model

In [100]:
from tensorflow.keras.applications import VGG16

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Input(shape=(224, 224, 3)))
# 
# model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256, activation='relu'))
# 
# 
# # Output
# model.add(tf.keras.layers.Dense(4, activation='softmax'))

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

model.summary()

Model: "functional_229"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_25 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 4)              │         1,02

 Total params: 27,692,612 (105.64 MB)

 Trainable params: 12,977,924 (49.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Compile the model

In [101]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

Train the model

In [102]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=3, 
    verbose=1
)

model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[stop_when_80, lr_scheduler]
)

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.3141 - loss: 3.7524 - val_accuracy: 0.3250 - val_loss: 4.9085 - learning_rate: 0.0010
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.2877 - loss: 5.2640 - val_accuracy: 0.3500 - val_loss: 4.0246 - learning_rate: 0.0010
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.3982 - loss: 3.8210 - val_accuracy: 0.3250 - val_loss: 1.5570 - learning_rate: 0.0010
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.4154 - loss: 2.8379 - val_accuracy: 0.6000 - val_loss: 0.9352 - learning_rate: 0.0010
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.4029 - loss: 2.0198 - val_accuracy: 0.7250 - val_loss: 0.8629 - learning_rate: 0.0010
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.4031 - loss: 1.5065 - val_accuracy: 0.5500 - val_loss: 0.9428 - learning_rate: 0.0010
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.3918 - loss: 1.4241 - val_accuracy: 0.6250 - val_l

KeyboardInterrupt: 

Evaluate the model

In [ ]:
model.evaluate(validation_generator)